In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window

df_pedido_total_silver = spark.table("medalhao.silver.pedido_total_silver")
df_consumidores_silver = spark.table("medalhao.silver.consumidores_silver")
df_pedidos_silver = spark.table("medalhao.silver.pedidos_silver")
df_itens_pedidos_silver = spark.table("medalhao.silver.pedidos_itens_silver")
df_produtos_silver = spark.table("medalhao.silver.produtos_silver")
df_vendedores_silver = spark.table("medalhao.silver.vendedores_silver")
df_avaliacao_silver = spark.table("medalhao.silver.avaliacoes_pedidos_validos_silver")
df_cotacao_silver = spark.table("medalhao.silver.cotacao_dolar_silver")

In [0]:
ft_vendas_consumidor_local = (df_pedido_total
                                .join
                                (ft_consumidores, on="id_consumidor")
                                .select
                                (
                                    F.col("id_consumidor"),
                                    F.col("id_pedido"),
                                    F.col("data_pedido"),
                                    F.col("valor_total_pago_brl").alias("valor_total_pedido_brl"),
                                    F.col("estado"),
                                    F.col("cidade"),
                                    F.col("data_pedido")
                                )
                            )

ft_vendas_consumidor_local.printSchema()

In [0]:
df_total_compras_por_consumidor = (ft_vendas_consumidor_local.
                                   groupBy("cidade", "estado")
                                   .agg(
                                        F.sum("valor_total_pedido_brl").alias("valor_total_localidade"),
                                        F.count("id_pedido").alias("quantidade_vendas")
                                   ))


display(df_total_compras_por_consumidor.limit(10))

df_total_compras_por_consumidor.createOrReplaceTempView("_total_compras_por_consumidor")

In [0]:
df_total_vendas_por_estado = spark.sql("SELECT estado, COUNT(quantidade_vendas) FROM view_total_compras_por_consumidor GROUP BY estado")
display(df_total_vendas_por_estado.limit(10))

In [0]:
df_consumidor_pedidos_itens = (
    df_consumidores
    .join(ft_pedidos, on="id_consumidor")
    .join(ft_itens_pedidos, on="id_pedido")
    .select(
        F.col("id_pedido"),
        F.col("id_consumidor"),
        F.col("id_vendedor"),
        F.col("entrega_no_prazo"),
        F.col("tempo_entrega_dias"),
        F.col("tempo_entrega_estimado_dias"),
        F.col("cidade"),
        F.col("estado")
    )
)

display(df_consumidor_pedidos_itens.limit(10))

In [0]:
df_tempo_medio_entrega_localidade = (df_consumidor_pedidos_itens.
                                     filter(F.col("entrega_no_prazo") != "Não entregue").
                                     groupBy("cidade", "estado").
                                      agg(
                                          F.avg("tempo_entrega_dias").alias("tempo_medio_entrega"),
                                          F.avg("tempo_entrega_estimado_dias").alias("tempo_medio_entrega_estimado")
                                      )
                                      .withColumn("entrega_maior_que_estimado", F.when(F.col("tempo_medio_entrega") > F.col("tempo_medio_entrega_estimado"), "Sim").otherwise("Não")))
df_tempo_medio_entrega_localidade.createOrReplaceTempView("view_tempo_medio_entrega_localidade")

In [0]:
df_vendedor_pontualidade = (df_consumidor_pedidos_itens
                            .groupBy("id_vendedor")
                            .agg(
                                F.count(F.when(F.col("entrega_no_prazo") == "Não", F.col("entrega_no_prazo"))).alias("total_atrasados"),
                                F.count(F.col("id_pedido")).alias("total_pedidos")
                            )
                            .withColumn("percentual_atraso", (F.col("total_atrasados") * 100 / F.col("total_pedidos")))
                            .withColumn("percentual_atraso", F.concat(F.col("percentual_atraso").cast("string"), F.lit("%")))
                            )
df_vendedor_pontualidade.createOrReplaceTempView("df_vendedor_pontualidade")

In [0]:
from pyspark.sql import functions as F

data_inicio = dbutils.widgets.get("data_inicio")
data_fim = dbutils.widgets.get("data_fim")

df = spark.sql(
    f"SELECT DATE('{data_inicio}') AS data_inicio, DATE('{data_fim}') AS data_fim"
)

dm_tempo = (
    df
    .withColumn("array_de_dias_a_adicionar", F.sequence(F.lit(1), F.date_diff("data_fim", "data_inicio")))
    .withColumn("dias_a_adicionar", F.explode("array_de_dias_a_adicionar"))
    .withColumn("sk_tempo", F.date_add("data_inicio", "dias_a_adicionar"))
    .withColumn("ano", F.year("sk_tempo"))
    .withColumn("trimestre", F.quarter("sk_tempo"))
    .withColumn("mes", F.month("sk_tempo"))
    .withColumn("semana_do_ano", F.weekofyear("sk_tempo"))
    .withColumn("dia", F.dayofmonth("sk_tempo"))
    .withColumn("dia_da_semana_num", F.dayofweek("sk_tempo"))
    .withColumn("dia_da_semana_nome", F.expr("""
        CASE dayofweek(sk_tempo)
            WHEN 1 THEN 'Domingo'
            WHEN 2 THEN 'Segunda-feira'
            WHEN 3 THEN 'Terça-feira'
            WHEN 4 THEN 'Quarta-feira'
            WHEN 5 THEN 'Quinta-feira'
            WHEN 6 THEN 'Sexta-feira'
            WHEN 7 THEN 'Sábado'
        END
    """))
    .withColumn("mes_nome", F.expr("""
        CASE month(sk_tempo)
            WHEN 1 THEN 'Janeiro'
            WHEN 2 THEN 'Fevereiro'
            WHEN 3 THEN 'Março'
            WHEN 4 THEN 'Abril'
            WHEN 5 THEN 'Maio'
            WHEN 6 THEN 'Junho'
            WHEN 7 THEN 'Julho'
            WHEN 8 THEN 'Agosto'
            WHEN 9 THEN 'Setembro'
            WHEN 10 THEN 'Outubro'
            WHEN 11 THEN 'Novembro'
            WHEN 12 THEN 'Dezembro'
        END
    """))
    .withColumn("eh_fim_de_semana", F.when((F.col("dia_da_semana_num") == 1) | (F.col("dia_da_semana_num") == 7), "Sim").otherwise("Não"))
    .drop("dias_a_adicionar", "data_fim", "array_de_dias_a_adicionar", "data_inicio")
)

display(dm_tempo)

In [0]:
dim_clientes_df = (df_consumidores_silver
                   .select(
                       F.col("id_consumidor").alias("id_cliente"),
                       F.col("cidade"),
                       F.col("prefixo_cep"),
                       F.col("estado")
                   ))

dim_produtos_df = (df_produtos_silver
                   .select(
                       F.col("id_produto").alias("id_produto"),
                       F.col("categoria_produto"),
                       F.col("comprimento_centimetros"),
                       F.col("altura_centimetros"),
                       F.col("largura_centimetros"),
                       F.col("peso_produto_gramas"),
                   ))

dim_vendedores_df = (df_vendedores_silver
                     .select(
                         F.col("id_vendedor").alias("id_vendedor"),),
                         F.col("cidade"),
                         F.col("estado"),
                        F.col("prefixo_cep"),
                     )

In [0]:
schema = T.StructType([
    T.StructField("sk_tempo", T.DateType(), True),
    T.StructField("ano", T.IntegerType(), True),
    T.StructField("trimestre", T.IntegerType(), True),
    T.StructField("mes", T.IntegerType(), True),
    T.StructField("semana_do_ano", T.IntegerType(), True),
    T.StructField("dia", T.IntegerType(), True),
    T.StructField("dia_da_semana_num", T.IntegerType(), True),
    T.StructField("dia_da_semana_nome", T.StringType(), True),
    T.StructField("mes_nome", T.StringType(), True),
    T.StructField("eh_fim_de_semana", T.StringType(), True)
])

dm_tempo = spark.createDataFrame([], schema)

data_inicio = "2022-12-31"
data_fim = "2023-12-31"

diff_days = F.date_diff(F.lit(data_inicio), F.lit(data_fim))

dm_tempo = dm_tempo.withColumn("array_to_add",F.lit(1) - diff_days)

In [0]:
dim_media_avaliacao = (
    df_pedido_total_silver
    .join(df_avaliacao_silver, on="id_pedido")
    .groupBy("id_pedido")
    .agg(
        F.avg("avaliacao").alias("media_avaliacao_pedido")
    )
)

display(dim_media_avaliacao.limit(10))

In [0]:
ft_vendas_geral = (
    df_itens_pedidos_silver
    .join(dim_media_avaliacao, on="id_pedido")
    .join(df_pedidos_silver, on="id_pedido")
    .join(df_pedido_total_silver, on="id_pedido")
    .join(df_cotacao_silver, F.to_date(df_pedidos_silver.data_pedido) == F.to_date(df_cotacao_silver.data_cotacao))
    .select(
        F.col("id_pedido"),
        F.col("id_item"),
        F.col("id_consumidor").alias("fk_cliente "),
        F.col("id_produto").alias("fk_produto"),
        F.col("id_vendedor").alias("fk_vendedor"),
        F.col("data_pedido").alias("fk_tempo"),
        F.col("status").alias("status_pedido"),
        F.col("tempo_entrega_dias"),
        F.col("entrega_no_prazo"),
        F.col("preco_BRL").alias("valor_produto_brl"),
        F.col("preco_frete").alias("valor_frete_brl"),
        F.col("cotacaoDolar").alias("cotacao_dolar"),
        F.col("media_avaliacao_pedido").alias("avaliacao_pedido")
    )
    .withColumn("valor_total_item_brl". F.col("preco_BRL") + F.col("preco_frete"))
    .withColumn("valor_produto_usd", F.col("preco_BRL")/ F.col("cotacao_dolar"))
    .withColumn("valor_frete_usd", F.col("preco_frete")/ F.col("cotacao_dolar"))
    .withColumn("valor_total_item_usd", F.col("valor_produto_usd") + F.col("valor_frete_usd"))
)

display(ft_vendas_geral.limit(10))